In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

ModuleNotFoundError: No module named 'yfinance'

In [1]:
#%conda install -c conda-forge yfinance

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\HP\anaconda3

  added / updated specs:
    - yfinance


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blosc-1.21.3               |       h6c2663c_0          86 KB
    certifi-2023.11.17         |     pyhd8ed1ab_0         155 KB  conda-forge
    conda-4.14.0               |   py39hcbf5309_0         1.0 MB  conda-forge
    frozendict-2.3.10          |   py39ha55989b_3          45 KB  conda-forge
    html5lib-1.1               |     pyh9f0ad1d_0          89 KB  conda-forge
    imagecodecs-2021.8.26      |   py39h319e4f4_2         6.1 MB
    jxrlib-1.1                 |       h8ffe710_2         620 KB  conda-forge
    libarchive-3.6.2           |       h2033e3e_1         1.8 MB
    libbrotlicommon-1.0.9      |       hcfcfb64_9          71 KB  conda-forge
    libbrotlidec-1.0.9         |       hcfc



==> WARNING: A newer version of conda exists. <==
  current version: 4.13.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda




peewee-3.17.0        | 354 KB    |            |   0% 
peewee-3.17.0        | 354 KB    | 4          |   5% 
peewee-3.17.0        | 354 KB    | ########## | 100% 
peewee-3.17.0        | 354 KB    | ########## | 100% 

libbrotlicommon-1.0. | 71 KB     |            |   0% 
libbrotlicommon-1.0. | 71 KB     | ##2        |  23% 
libbrotlicommon-1.0. | 71 KB     | ########## | 100% 
libbrotlicommon-1.0. | 71 KB     | ########## | 100% 

jxrlib-1.1           | 620 KB    |            |   0% 
jxrlib-1.1           | 620 KB    | 2          |   3% 
jxrlib-1.1           | 620 KB    | #########  |  90% 
jxrlib-1.1           | 620 KB    | ########## | 100% 

libxml2-2.10.4       | 2.8 MB    |            |   0% 
libxml2-2.10.4       | 2.8 MB    | 7          |   7% 
libxml2-2.10.4       | 2.8 MB    | ##9        |  29% 
libxml2-2.10.4       | 2.8 MB    | ####8      |  48% 
libxml2-2.10.4       | 2.8 MB    | ######7    |  67% 
libxml2-2.10.4       | 2.8 MB    | ########7  |  88% 
libxml2-2.10.4       | 2.

In [ ]:
$ conda update -n base -c defaults conda

In [ ]:
tickers = ['AMZN', 'MSFT', 'NVDA']

dfs = []

for ticker in tickers:
    df = yf.download(ticker).reset_index()
    df['range'] = df['High']/df['Low'] -1
    df['open_to_close'] = df['Close']/df['Open'] -1
    df['ticker'] = ticker
    
    dfs.append(df)
    
df = pd.concat(dfs).reset_index(drop = True)

### 2. Feature Reshaping

* Time series algos require data in the shape =  (example #, feature length, feature #)


* Also, I assume that adjacent examples split apart by one day will be similar - therefore, let's undersample by selecting every   5th row.

  This may save us some training time and still retain most of the variance in the examples.
  


In [ ]:
feature_length = 50
feature_cols = ['Close', 'range', 'open_to_close']

def time_series(df, col, name):
    '''
    form the lagges column fo this feature
    '''
    return df.assign(**{
        f'{name}_t-{lag}': col.shift{lag}
        for lag in range(0, feature_length)
    })

feature_dfs = []

for ticker in tickers:
    df_ticker = df[df['ticker'] == ticker].copy()
    
    for col in feature_cols:
        df_ticker = time_series(df_ticker, df_ticker[col], f'feat_{col})
        
    #if any infinite value slips in, replace them with nan and drop all nan rows
    df_ticker = df_ticker.replace({-np.inf:np.nan, np.inf:np.nan})
    df_ticker = df_ticker.dropna().reset_index(drop = True)
                                
    #since adjoining days will have similar pattern, lets keep only every fifth case
    df_ticker = df_ticker.iloc[::5]
                                
    feature_dfs.append(df_ticker)
                                
feature_df = pd.concat(feature_dfs)
                                
print('Number of samples in the entire dataset: ', feature_df.shape[o])

feature_df.head()

* With all these new cols, we now need to reshape into a 3D array

  This is achieved with the attached code

  I will say, this whole reshaping nonsense is tricky to wrap your head around! So don't worry if it's not immediately obvious    🙂